## Question Answering over Documents with Xinference and LangChain


This demo walks through how to build an LLM-driven question-answering application with Xinference and LangChain.

## Deploy Xinference Locally or in a Distributed Cluster.

For local deployment, run `xinference`. 

To deploy Xinference in a cluster, first start an Xinference supervisor using the `xinference-supervisor`. You can also use the option -p to specify the port and -H to specify the host. The default port is 9997.

Then, start the Xinference workers using `xinference-worker` on each server you want to run them on. 

You can consult the README file from [Xinference](https://github.com/xorbitsai/inference) for more information.
## Start a Model

To use Xinference with LangChain, you need to first launch a model. You can use command line interface (CLI) to do so:

In [44]:
!xinference launch -n llama-2 -q q4_0

Model uid: 7cfc3c22-2ac7-11ee-b3ae-d29396a3f064


## Prepare the Documents

In [53]:
from langchain.document_loaders import TextLoader
# 
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("/Users/nijiayi/langchain/docs/extras/modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


## Set Up an Embedding Model

In [54]:
from langchain.embeddings import XinferenceEmbeddings

embeddings = XinferenceEmbeddings(
    server_url="http://0.0.0.0:9997",
    model_uid = "7cfc3c22-2ac7-11ee-b3ae-d29396a3f064"
)

In [56]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)


In [58]:
query = "What did the president say about Ketanji Brown Jackson"

docs = db.similarity_search(query)

print(docs[2].page_content)

And built the strongest, freest, and most prosperous nation the world has ever known. 

Now is the hour. 

Our moment of responsibility. 

Our test of resolve and conscience, of history itself. 

It is in this moment that our character is formed. Our purpose is found. Our future is forged. 

Well I know this nation.  

We will meet the test. 

To protect freedom and liberty, to expand fairness and opportunity. 

We will save democracy. 

As hard as these times have been, I am more optimistic about America today than I have been my whole life. 

Because I see the future that is within our grasp. 

Because I know there is simply nothing beyond our capacity. 

We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. 

The only nation that can be defined by a single word: possibilities. 

So on this night, in our 245th year as a nation, I have come to report on the State of the Union.


In [47]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)


In [49]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [52]:
print(docs[2].page_content)



And built the strongest, freest, and most prosperous nation the world has ever known. 

Now is the hour. 

Our moment of responsibility. 

Our test of resolve and conscience, of history itself. 

It is in this moment that our character is formed. Our purpose is found. Our future is forged. 

Well I know this nation.  

We will meet the test. 

To protect freedom and liberty, to expand fairness and opportunity. 

We will save democracy. 

As hard as these times have been, I am more optimistic about America today than I have been my whole life. 

Because I see the future that is within our grasp. 

Because I know there is simply nothing beyond our capacity. 

We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. 

The only nation that can be defined by a single word: possibilities. 

So on this night, in our 245th year as a nation, I have come to report on the State of the Union.


## Connect to the Vector Database

For vector store, we use the Milvus vector database. [Milvus](https://milvus.io/docs/overview.md) is a database that stores, indexes, and manages massive embedding vectors generated by deep neural networks and other machine learning models. To run, you should first [Install Milvus Standalone with Docker Compose](https://milvus.io/docs/install_standalone-docker.md).

In [6]:



vector_db = Milvus.from_documents(
    docs,
    xinference_embeddings,
    connection_args={"host": "0.0.0.0", "port": "19530"},
)

In [36]:
print(docs)

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.', metadata={'source'

In [43]:
query_result = xinference_embeddings.embed_query("What did the president say about Ketanji Brown Jackson")
print(query_result)

[2.7027578353881836, -1.2457365989685059, -0.2186949998140335, -0.09360531717538834, -1.064269781112671, -0.34095239639282227, 1.0225869417190552, -4.581243515014648, 0.993728756904602, 0.500874936580658, 0.6742779612541199, -1.7945033311843872, -0.4607464671134949, 3.2834622859954834, -1.4134358167648315, -2.810467481613159, -4.379144668579102, 1.9699015617370605, 0.8201614618301392, -3.4372918605804443, 3.1795616149902344, -1.4286712408065796, 0.6034550070762634, 1.180527925491333, -2.765810012817383, 0.18189167976379395, 0.47852909564971924, 0.8118906617164612, 3.2079362869262695, 1.9850454330444336, 0.3195192217826843, -0.02779005467891693, 0.20471210777759552, 3.581491470336914, 0.8007687330245972, 0.26625025272369385, -4.487994194030762, 1.570480227470398, 0.7177330851554871, -0.6060148477554321, -0.4855101406574249, -1.205269694328308, 2.3347525596618652, 1.9310234785079956, 0.0810767263174057, 0.9219681620597839, 1.07597815990448, 0.9397372007369995, -0.10947447270154953, -0.63

In [40]:
print(xinference_embeddings.embed_documents(docs))

TypeError: Object of type Document is not JSON serializable

In [28]:
query = "How many days ago did Vladimir Putin speak"
docs = vector_db.similarity_search(query)

In [19]:
print(docs)

[Document(page_content='We’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  \n\nAnd tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. \n\nBy the end of this year, the deficit will be down to less than half what it was before I took office.  \n\nThe only president ever to cut the deficit by more than one trillion dollars in a single year. \n\nLowering your costs also means demanding more competition. \n\nI’m a capitalist, but capitalism without competition isn’t capitalism. \n\nIt’s exploitation—and it drives up prices. \n\nWhen corporations don’t have to compete, their profits go up, your prices go up, and small businesses and family farmers and ranchers go under. \n\nWe see it happening with ocean carriers moving goods in and out of America. \n\nDuring the pandemic, these foreign-owned companies raised prices by as much as 1,000% and made record profits.', metadata=

In [29]:
docs[0].page_content


'And my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today. \n\nNow is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.'

In [42]:
print(xinference_embeddings.embed_query('And my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today. \n\nNow is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.'
))

[0.7053864002227783, 0.8866159915924072, 0.8066064715385437, -0.3436307907104492, -0.7952313423156738, -1.9158947467803955, 1.230029582977295, 2.061551809310913, -0.42197099328041077, 0.5758300423622131, 3.493018388748169, -2.5552120208740234, -1.6168875694274902, 3.5874342918395996, 0.9325556755065918, -1.3997396230697632, 2.3210864067077637, 0.7876622080802917, 1.4212266206741333, -1.8164407014846802, 2.9597771167755127, -0.8429591059684753, -0.2797074317932129, 0.6953061819076538, -0.5377737283706665, 1.5053642988204956, 1.0559496879577637, -0.36977121233940125, 0.42265886068344116, 1.8783578872680664, -0.770344078540802, -1.2769765853881836, -1.4999932050704956, 2.2529077529907227, -0.5758848190307617, -0.12591026723384857, 0.2484527975320816, 2.5403411388397217, 0.4164864122867584, -1.4605110883712769, -0.6903164982795715, 0.14758101105690002, -2.2901687622070312, -0.08506764471530914, -0.27971792221069336, 1.2221777439117432, -0.6633301377296448, 2.085334539413452, -0.96230757236

In [30]:
docs[1].page_content

'And built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.  \n\nWe will meet the test. \n\nTo protect freedom and liberty, to expand fairness and opportunity. \n\nWe will save democracy. \n\nAs hard as these times have been, I am more optimistic about America today than I have been my whole life. \n\nBecause I see the future that is within our grasp. \n\nBecause I know there is simply nothing beyond our capacity. \n\nWe are the only nation on Earth that has always turned every crisis we have faced into an opportunity. \n\nThe only nation that can be defined by a single word: possibilities. \n\nSo on this night, in our 245th year as a nation, I have come to report on the State of the Union.'

In [31]:
docs[2].page_content

'We’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  \n\nAnd tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. \n\nBy the end of this year, the deficit will be down to less than half what it was before I took office.  \n\nThe only president ever to cut the deficit by more than one trillion dollars in a single year. \n\nLowering your costs also means demanding more competition. \n\nI’m a capitalist, but capitalism without competition isn’t capitalism. \n\nIt’s exploitation—and it drives up prices. \n\nWhen corporations don’t have to compete, their profits go up, your prices go up, and small businesses and family farmers and ranchers go under. \n\nWe see it happening with ocean carriers moving goods in and out of America. \n\nDuring the pandemic, these foreign-owned companies raised prices by as much as 1,000% and made record profits.'